# Check ICA
The main purpose of the code in this notebook is to:
1) preprocess the raw data (high and low pass filter, exclude bad channels, interpolate bad channels)
2) identify noise components in the ICA solutions generated in 'run_ica.py'
3) removing noise components and applying ICA to the data
4) epoching the data and saving it to disk

To move the epochs to source space use the file 'epochs_2_source_space.py'

In [ ]:
import mne
import json
import os

# loading in the txt file with the channels that should be labeled as bad channels
with open('../info_files/session_info.txt', 'r') as f:
    file = f.read()
    session_info = json.loads(file)

In [ ]:
session = 'visual_05'

### RAW DATA ###
filepath_raw = os.path.join('/media', 'raw_data', 'franscescas_data', 'raw_data', f'{session}.fif')

# Loading in the raw data
raw = mne.io.read_raw_fif(filepath_raw, on_split_missing='warn');
raw.load_data();
raw.pick_types(meg=True, eeg=False, stim=True)

### EXCLUDING BAD CHANNELS ###
# using dict[] notation to get the bad channels for the specific file. Not using dict.get() as this does not raise a key-error if the key does not exist
bad_channels_file = session_info[session + '.fif']['bad_channels']

# marking the channels as bad
raw.info['bads'] = bad_channels_file


# cropping beginning and end of recording
tmin = session_info[session + '.fif']['tmin']
tmax = session_info[session + '.fif']['tmax']

cropped = raw.copy().crop(tmin = tmin, tmax = tmax)
del raw

### HIGH PASS FILTERING ###
filt_raw = cropped.copy().filter(l_freq=1, h_freq=40)
del cropped

filt_raw.interpolate_bads(origin=(0, 0, 0.04)) 

### ICA ###
ica_filename = f'/media/8.1/intermediate_data/laurap/ica/ica_solution/{session}-ica.fif'
ica = mne.preprocessing.read_ica(ica_filename)
ica.apply(filt_raw)

In [ ]:
ica.plot_sources(filt_raw);

In [ ]:
ica.plot_components()

In [ ]:
with open('../info_files/session_info.txt', 'r') as f:
    file = f.read()
    session_info = json.loads(file)

info_ses = session_info[session + '.fif']['noise_components']
noise_components = info_ses

ica.plot_properties(filt_raw, picks = noise_components);

In [ ]:
ica.exclude = noise_components
ica.apply(filt_raw)

### Epoching

In [ ]:
#  creating the epochs for the ICA data
events = mne.find_events(filt_raw, shortest_event=1)
reject = dict(grad=4000e-13, mag=4e-12)  

# event ids
with open('../info_files/event_ids.txt', 'r') as f:
    file = f.read()
    event_ids = json.loads(file)
    
epochs = mne.Epochs(filt_raw, events, event_ids, tmin=0, tmax=1, proj=True, baseline=None, preload=True, reject=reject, on_missing = 'warn')

outpath = f'/media/8.1/final_data/laurap/epochs/{session}-epo.fif'
epochs = epochs.resample(250)
epochs.save(outpath, overwrite = True)